### Primer Design
This notebook defines a funciton to search for the melting temperature of many pairs of duplexes by performing a binary search over results from the parallel `concentrations` application.

The following code then uses that function to search for complementary primer pairs with a desired melting temperature.

In [1]:
%pylab notebook
import pandas as pd
import cupyck
import nupyck

Populating the interactive namespace from numpy and matplotlib


In [2]:
def melting_temp(session, jobs):
    
    jobs = jobs.join(
        pd.DataFrame([
            { "sequences"        : [job.sequence_1, job.sequence_2],
              "temperature"      : 50,
              "t_hi"             : 100,
              "t_lo"             : 0,
              "x0"               : np.array([1e-6, 1e-6]),
              "max_complex_size" : 2,
              "converged"        : False
            }
            for job in jobs.itertuples()
        ])
    )
    
    in_progress = jobs
    
    while not jobs.converged.all():

        results = session.concentrations(in_progress)
        
        duplex_yield = results.apply(
            lambda row:
              row.concentrations[1,2] / row.x0[0],
            axis = 1
        )
        
        def update(job):
            
            if job.converged:
                return job

            temp, t_hi, t_lo = job[['temperature', 't_hi', 't_lo']]
            
            yld = duplex_yield.loc[job.name]
            
            if yld >= 0.51:
                job.temperature = temp + (t_hi - temp) / 2.0
                job.t_lo = temp

            elif yld <= 0.49:
                job.temperature = temp - (temp - t_lo) / 2.0
                job.t_hi = temp
            
            else:
                job.converged = True
            
            return job
              
        jobs = jobs.apply(update, axis=1)
        
        in_progress = jobs[~jobs.converged]
        print "%d / %d complete" % (len(jobs) - len(in_progress), len(jobs))

    jobs.rename(columns={"temperature": "melting_temp"}, inplace=True)
    
    return jobs[["sequence_1", "sequence_2", "melting_temp"]]    

In [3]:
def randseq(n):
    return "".join(np.random.choice(list("ATCG"), n))

In [4]:
jobs = pd.DataFrame([
    { "sequence_1": seq,
      "sequence_2": nupyck.revcomp(seq)
    }
    for seq in [randseq(20) for _ in range(5000)]
])

In [5]:
session = cupyck.GPUSession(max_seqlen = 50)

In [6]:
results = melting_temp(session, jobs)

0 / 5000 complete
86 / 5000 complete
97 / 5000 complete
201 / 5000 complete
394 / 5000 complete
758 / 5000 complete
1526 / 5000 complete
3097 / 5000 complete
5000 / 5000 complete


In [7]:
results[(69 < results.melting_temp) & (results.melting_temp < 71) ]

,sequence_1,sequence_2,melting_temp
1,AAATTCCATAGAATTGCCTC,GAGGCAATTCTATGGAATTT,69.140625
17,TTCACCTGTTGTGATTTCTC,GAGAAATCACAACAGGTGAA,70.703125
19,CTGGGTACTCATTTTCCAAA,TTTGGAAAATGAGTACCCAG,69.140625
23,TAAAGAAGCTAGTTTGACCA,TGGTCAAACTAGCTTCTTTA,70.312500
53,CGCTTACTACCATTAGTGCC,GGCACTAATGGTAGTAAGCG,70.898438
62,GGGTAGTTTTCGTCTATCGT,ACGATAGACGAAAACTACCC,70.117188
76,AGTGTTTGCAGAACGTAAAC,GTTTACGTTCTGCAAACACT,70.507812
96,GCCCATAGTAAAAGGAGTTA,TAACTCCTTTTACTATGGGC,70.312500
106,TTAGACTAGCACTCTTCTCC,GGAGAAGAGTGCTAGTCTAA,70.898438
115,CATCGTAGAGACAAAACCAA,TTGGTTTTGTCTCTACGATG,70.703125
